In [1]:
import os
from netml.pparser.parser import PCAP
import pprint
from scapy.all import *
from scapy.layers.inet import *
from tqdm import tqdm

import tensorflow as tf


In [2]:
dataDir = "/code/2.Malwares/"
targetFile = "Zeus.pcap"
destination = dataDir + "out"
tsharkPath="/usr/bin/tshark"
packetSizeInBytes = 100
packetsPerFlow = 2
tfrecorsFolder = dataDir + 'tfrecords/'
testFolder = dataDir + 'test/'
validationFolder = dataDir + 'validation/'

nb_shards = 10
arrOfTfrecordsWriter = [] # un objet criter par shard

In [3]:
cats = ["Benign","Malware"] #Catégorie
listofProcessedFiles = []
training_data = []

In [4]:
for i in range(nb_shards):
    arrOfTfrecordsWriter.append(tf.io.TFRecordWriter(path=tfrecorsFolder + 'shard_' + str(i) + '.tfrecords'))

In [5]:
def getLabelForFile(filename):
    filename.replace('.pcap','')
    if filename.find('-') != -1:
        filename = filename.split('-')[0]
    for index,processedFile in enumerate(listofProcessedFiles):
        if processedFile == filename:
            return index
    listofProcessedFiles.append(filename)
    return len(listofProcessedFiles)
    
#pprint.pprint(listofProcessedFiles)
#label = getLabelForFile('tes-test')
#print(label)

In [6]:
def _bytes_feature(value):# peut-être un tableau
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()  # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [7]:
def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [8]:
# Create the features dictionary.
def packet_example(packetArray, label):
    feature = {
        'label': _int64_feature(label),
        'packet1': _bytes_feature(packetArray[0]),
        'packet2': _bytes_feature(packetArray[1]),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

In [9]:
 def processPackets(packets):
    newFlow = []
    for i,packet in enumerate(packets):
        packetHex = linehexdump(packet.payload,onlyhex=1,dump=True) # dump packet to hex 01 DE 
        packetHexnoSpace = packetHex.replace(' ','').lower() #to hex 01de
        packetBytes = bytes.fromhex(packetHexnoSpace) # to bytestring
        if len(packetBytes) < packetSizeInBytes: # ajuster les packets à une taille de 100 bytes 
            packetPadded = packetBytes.ljust(packetSizeInBytes,b'\0') #padding
        else:
            packetPadded = packetBytes[:packetSizeInBytes] # couper les paquet trop grand
        if i >= packetsPerFlow:
            break
        newFlow.append(packetPadded)
    return newFlow


In [10]:
import random
for label ,cat in enumerate(cats): # pour chaque dossier Benign, Malware
    path = os.path.join(dataDir,cat)
    for filePcap in os.listdir(path): #parcours les fichiers pour chaque dossier
        pprint.pprint(path)
        pprint.pprint(filePcap)

        parsedPcap = PCAP(
            os.path.join(path,filePcap),
            flow_ptks_thres=packetsPerFlow,
            verbose=3
        )
        parsedPcap.pcap2flows(interval=1000)
        for flow in tqdm(parsedPcap.flows): #(val,182.168.1.15,,)(<Ether
            flow = flow[1:] # retirer le premier élément car inutile
            for packets in flow:
                writer = random.choice(arrOfTfrecordsWriter)
                tf_example = packet_example(processPackets(packets), label)
                writer.write(tf_example.SerializeToString())
for writer in arrOfTfrecordsWriter:
    writer.close()


'/code/2.Malwares/Benign'
'Outlook.pcap'
'_pcap2flows()' starts at 2021-05-03 13:16:53
pcap_file: /code/2.Malwares/Benign/Outlook.pcap


100%|██████████| 15/15 [00:00<00:00, 3193.47it/s]

After splitting flows, the number of subflows: 15 and each of them has at least 2 packets.
'_pcap2flows()' ends at 2021-05-03 13:16:58 and takes 0.0783 mins.
'/code/2.Malwares/Benign'
'WorldOfWarcraft.pcap'
'_pcap2flows()' starts at 2021-05-03 13:16:58
pcap_file: /code/2.Malwares/Benign/WorldOfWarcraft.pcap



  4%|▎         | 572/15730 [00:00<00:02, 5718.61it/s]

After splitting flows, the number of subflows: 15730 and each of them has at least 2 packets.
'_pcap2flows()' ends at 2021-05-03 13:17:40 and takes 0.6998 mins.


...

100%|██████████| 16585/16585 [00:02<00:00, 7185.15it/s]


'/code/2.Malwares/Malware'
'Neris.pcap'
'_pcap2flows()' starts at 2021-05-03 13:47:14
pcap_file: /code/2.Malwares/Malware/Neris.pcap


  1%|          | 421/43788 [00:00<00:10, 4199.38it/s]

After splitting flows, the number of subflows: 43788 and each of them has at least 2 packets.
'_pcap2flows()' ends at 2021-05-03 13:50:16 and takes 3.0439 mins.


100%|██████████| 43788/43788 [00:09<00:00, 4785.25it/s]


In [11]:
tfrecordsPath = os.path.join(tfrecorsFolder)
listtfRecordsFile = os.listdir(tfrecordsPath)
validationFiles = random.sample(listtfRecordsFile,2)
pprint.pprint(validationFiles)
os.rename(tfrecordsPath + validationFiles[0],validationFolder + validationFiles[0])
os.rename(tfrecordsPath + validationFiles[1],testFolder + validationFiles[1])

['shard_4.tfrecords', 'shard_3.tfrecords']
